# MAS Thesis Notebook
## Krishna Dave (UID: 905636874)

## Description
This notebook contains example code for privacy auditing using the TAPAS toolbox of synthetic data generated from the Breast Cancer Wisconsin Diagnostic Data (https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic). This toolbox is used for evaluating the privacy of synthetic data using adversarial techniques. The code for the TAPAS toolbox is from: https://github.com/alan-turing-institute/tapas 

## Setup Instructions
1. Install poetry (system-wide) from https://python-poetry.org/docs/ 
2. It can also be installed using pip: `pip install git+https://github.com/alan-turing-institute/privacy-sdg-toolbox`
3. Activate virtual environment inside the Jupyter notebook using `poetry shell`
4. Add the virtual environment to the available kernels for the notebook.
5. Make sure that this notebook is located in the correct directory inside the wider directory to ensure the correctness of the relative imports and file paths.

### Setup: Installation of other packages

In [2]:
!pip install be-great
!pip install --upgrade pip --quiet
!pip install --upgrade ctgan --quiet
!pip install sdv opacus autodp smartnoise-synth --quiet

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple


In [21]:
!pip install openai

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0


In [3]:
!pip list

Package                       Version     Editable project location
----------------------------- ----------- ----------------------------------------------------------------------------------------------------------------
absl-py                       1.3.0
accelerate                    0.25.0
aiohttp                       3.9.1
aiosignal                     1.3.1
alabaster                     0.7.12
altgraph                      0.17.3
annotated-types               0.6.0
antlr4-python3-runtime        4.9.3
anyio                         3.6.2
appnope                       0.1.3
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.2.0
astunparse                    1.6.3
async-timeout                 4.0.3
attrs                         22.1.0
autodp                        0.2.3.1
Babel                         2.10.3
backcall                      0.2.0
be-great                      0.0.7
beautifulsoup4                4.11.1
black      

## Importing necessary packages

In [2]:
# Importing standard packages
import numpy as np
import pandas as pd
import time
import json
import itertools
from enum import Enum
import os

# Importing from tapas
import tapas.datasets
import tapas.generators
import tapas.attacks
import tapas.threat_models
import tapas.report
from tapas.generators import Generator, ReprosynGenerator, Raw
import tqdm
from modules.myctgan import CTGAN
from modules.myctgan import DPCTGAN
from modules.myctgan import PATEGAN

# Importing from sklearn
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer

# Importing from mbi and be_great
from mbi import Dataset, Domain
from be_great import GReaT

# Importing openai to call GPT4 API from OpenAI
from openai import OpenAI

/Users/krishnadave/Library/Caches/pypoetry/virtualenvs/tapas-T4rVcRt4-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/krishnadave/Library/Caches/pypoetry/virtualenvs/tapas-T4rVcRt4-py3.9/lib/python3.9/site-packages/mbi/__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')
2023-12-27 17:24:17.514818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Using the GReaT framework API
### Description
Generates Realistic Synthetic Tabular data with pretrained Transformer-based language models. Here is an example that uses their API to generate synthetic data for the breast cancer dataset.

### Debugging Tips/Links
LLM model in the GReaT API has to be run on a GPU, it doesn't work on a Mac CPU. 

Otherwise, you will encounter the following errors: 
`RuntimeError: CUDA Out of memory` 
`RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1`

To solve this, run this on Google Colab, navigate to Edit > Notebook settings > select Runtime type to be Python 3 > Hardware accelerator to T4 GPU or paid A100 GPU/V100 GPU.

For the error: 
`An error has occurred: Breaking the generation loop!`
`To address this issue, consider fine-tuning the GReaT model for an longer period. This can be achieved by increasing the number of epochs.
Alternatively, you might consider increasing the max_length parameter within the sample function. For example: model.sample(n_samples=10, max_length=2000)
If the problem persists despite these adjustments, feel free to raise an issue on our GitHub page at: https://github.com/kathrinse/be_great/issues`

To solve this error, try passing in `fp16=true` in the model definition, increasing the `max_length` value and increasing the number of `epochs`.

Here are some links that helped me with debugging: 
- https://stackoverflow.com/questions/68166721/pytorch-fails-with-cuda-error-device-side-assert-triggered-on-colab
- https://discuss.pytorch.org/t/cuda-error-device-side-assert-triggered-cuda-kernel-errors-might-be-asynchronously-reported-at-some-other-api-call-so-the-stacktrace-below-might-be-incorrect-for-debugging-consider-passing-cuda-launch-blocking-1/160825/5
- https://github.com/pytorch/pytorch/issues/75534
- https://stackoverflow.com/questions/70340812/how-to-install-pytorch-with-cuda-support-with-pip-in-visual-studio
- https://medium.com/@snk.nitin/how-to-solve-cuda-out-of-memory-error-850bb247cfb2
- https://stackoverflow.com/questions/64589421/packagesnotfounderror-cudatoolkit-11-1-0-when-installing-pytorch
- https://github.com/pytorch/pytorch/issues/30664
- https://github.com/kathrinse/be_great/issues/42
- https://github.com/kathrinse/be_great/issues/40
- https://research.google.com/colaboratory/faq.html#gpu-availability

In [4]:
# Generates synethetic data using the GReaT framework's API
def generateSyntheticDataFromGReaT(batchSize: int, epochs: int, nSamples: int):
    # Loading the breast cancer dataset as a frame
    breastCancerData = load_breast_cancer(as_frame=True).frame
    
    # Defining an LLM model using the GReaT API
    model = GReaT(llm='distilgpt2', batch_size=batchSize, epochs=epochs)
    
    # Fitting the generated model to the breastCancerDataSet
    model.fit(breastCancerData)
    
    # Sampling the model to generate the synethic data
    synthetic_data = model.sample(n_samples=nSamples)
    
    # Returning the produced and sampled synthetic data
    return synthetic_data

In [4]:
# Function call to generate the synthetic data using GReaT; need to run this only once to generate the data, takes a while to generate 
# synthetic_data = generateSyntheticDataFromGReaT(batch_size=32, epochs=100, nSamples=100)

## Using OpenAI's GPT4 to generate synthetic data from real dataset
### Description
We can use OpenAI's GPT4 to generate synthetic data from real dataset using the API or chat. Following are examples of both.

### Useful Links
- https://github.com/openai/openai-cookbook
- https://github.com/openai/openai-python
- https://www.reddit.com/r/OpenAI/comments/161cygf/gpt4_api_access/?rdt=53543
- https://platform.openai.com/docs/introduction
- https://platform.openai.com/playground
- https://platform.openai.com/docs/api-reference
- https://platform.openai.com/api-keys
- https://platform.openai.com/docs/api-reference/authentication
- https://community.openai.com
- https://platform.openai.com/usage

### GPT4 API Example

In [ ]:
from pathlib import Path

# While you can provide an api_key keyword argument, we recommend using python-dotenv to add OPENAI_API_KEY="My API Key" to your .env file so that your API Key is not stored in source control.
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"), # Unique API Keys for paid GPT4 API usage can be created here to track : https://platform.openai.com/api-keys
)

# Request parameters that correspond to file uploads can be passed as bytes, a PathLike instance or a tuple of (filename, contents, media type).
client.files.create(
    file=Path("../data/BreastCancerRealDataset.csv"),
    purpose="synthetic data creation",
) 
# #The async client uses the exact same interface. If you pass a PathLike instance, the file contents will be read asynchronously automatically.

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Can you create synthetic data that mimics this real dataset, and output a csv file with that data.",
        }
    ],
    model="gpt4",
)

### ChatGPT 4 Example

<img src="../data/gpt4.png" alt="Alternative text" />

## Data Pre-processing: Labeling the Data Files

### Description
For data pre-processing, I had to label the raw and the generated data files with column names and indices since those were not already present in the raw data downloaded from the source.

The column names for the data files include:
1. ID (Patient ID)
2. DIAGNOSIS (M: Malignant, B: Benign)
3. RADIUS1
4. TEXTURE1
5. PERIMETER1
6. AREA1
7. SMOOTHNESS1
8. COMPACTNESS1
9. CONCAVITY1
10. CONCAVE_POINTS1
11. SYMMETRY1
12. FRACTAL_DIMENSION1
13. RADIUS2
14. TEXTURE2
15. PERIMETER2
16. AREA2
17. SMOOTHNESS2
18. COMPACTNESS2
19. CONCAVITY2
20. CONCAVE_POINTS2
21. SYMMETRY2
22. FRACTAL_DIMENSION2
23. RADIUS3
24. TEXTURE3
25. PERIMETER3
26. AREA3
27. SMOOTHNESS3
28. COMPACTNESS3
29. CONCAVITY3
30. CONCAVE_POINTS3
31. SYMMETRY3
32. FRACTAL_DIMENSION3

Ten real-valued features are computed for each cell nucleus in the dataset:
	a) radius (mean of distances from center to points on the perimeter)
	b) texture (standard deviation of gray-scale values)
	c) perimeter
	d) area
	e) smoothness (local variation in radius lengths)
	f) compactness (perimeter^2 / area - 1.0)
	g) concavity (severity of concave portions of the contour)
	h) concave points (number of concave portions of the contour)
	i) symmetry 
	j) fractal dimension ("coastline approximation" - 1)

## Loading Data

### Description
Loading data from the input csvPath, assigning it to the appropriate data types and returning a dataframe as a data type dictionary.

In [9]:
# Loads data from the input csvPath, assigns it to appropriate data types and returns a dataframe as data type dictionary
def loadBreastCancerData(csvPath: str):
    # Reads the csv file using the input csvPath
    df = pd.read_csv(csvPath, index_col=None)
    
    # Stores the column names of the DataFrame df in the variable cols
    cols = df.columns
    
    # Uses dictionary comprehension to set the data type of the columns to data types of str or float64
    dtype_dict = {"ID": "str", **{col: 'str' for col in cols[1:3]}, **{col: 'float64' for col in cols[3:]}}
    
    # Applies the data type conversions specified in dtype_dict to the DataFrame df
    # The astype() function is used to cast the columns of the DataFrame to specified data types
    df = df.astype(dtype_dict)
    
    return df

In [17]:
# Paths to the synthetic data files
breastCancerData_real = "../data/breastcancer_real_labeled2.csv"
breastCancerData_synthetic = "../data/breastcancer_synthetic_labeled2.csv"

# Callings loadBreastCancerData to load breast cancer data into a data frame
df = loadBreastCancerData(breastCancerData_real)

In [18]:
df.head()

,ID,DIAGNOSIS,RADIUS1,TEXTURE1,PERIMETER1,AREA1,SMOOTHNESS1,COMPACTNESS1,CONCAVITY1,CONCAVE_POINTS1,...,RADIUS3,TEXTURE3,PERIMETER3,AREA3,SMOOTHNESS3,COMPACTNESS3,CONCAVITY3,CONCAVE_POINTS3,SYMMETRY3,FRACTAL_DIMENSION3
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [19]:
df.describe()

,TEXTURE1,PERIMETER1,AREA1,SMOOTHNESS1,COMPACTNESS1,CONCAVITY1,CONCAVE_POINTS1,SYMMETRY1,FRACTAL_DIMENSION1,RADIUS2,...,RADIUS3,TEXTURE3,PERIMETER3,AREA3,SMOOTHNESS3,COMPACTNESS3,CONCAVITY3,CONCAVE_POINTS3,SYMMETRY3,FRACTAL_DIMENSION3
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [11]:
# Creating TabularDataset object
data = tapas.datasets.TabularDataset(data=df, 
                                     description=tapas.datasets.DataDescription(json.load(open("../data/cancer.json"))))

## Attacks, Threat Models and Privacy Auditing

### Description
Contains helper functions and methods for defining attacker knowledge on data, attacker knowledge on generator, threat models, different attack types provided in the TAPAS toolbox, generators for synthetic data, creation of target indices, functions that define the attack, train/test the model and generate metrics and reports.

In [14]:
# Enum for attacker knowledge on data
class AttackerKnowledgeOnData(Enum):
    # This attacker knowledge assumes access to some auxiliary dataset from which training datasets are sampled, as random subset of this auxiliary data.
    # A distinct testing dataset, sampled from the same distribution, is also used to generate testing samples.
    AuxiliaryDataKnowledge = 1
    
    # Also called worst-case attack, this assumes that the attacker knows the exact dataset used to generate 
    ExactDataKnowledge = 2
    
# Helper function that returns attacker knowledge on data
def getAttackerKnowledgeOnData(dataset, attackerKnowledgeOnData):
    if attackerKnowledgeOnData == AttackerKnowledgeOnData.AuxiliaryDataKnowledge:
        return tapas.threat_models.AuxiliaryDataKnowledge(dataset,
                        auxiliary_split=0.5, num_training_records=100, )
    elif attackerKnowledgeOnData == AttackerKnowledgeOnData.ExactDataKnowledge:
        return tapas.threat_models.ExactDataKnowledge(dataset, num_training_records=100, )

In [15]:
# Enum for attacker knowledge on generator
class AttackerKnowledgeOnGenerator(Enum):
    # BlackBoxKnowledge: The attacker has access to the generator method with access to the generator has an exact black-box.
    # The attacker can call the generator with the same parameters as were used to produce the real dataset. This is the recommended assumption on attacker knowledge.
    BlackBoxKnowledge = 1
    
    # The attacker does not have access to the generator. The attacker cannot call the generator, and the .generate method thus fails in training mode.
    # A generator is still needed to generate evaluation samples.
    NoBoxKnowledge = 2
    
    # The attacker has uncertain knowledge of the generator: they have access to the code, but not to some "parameters" of the code. 
    # Instead, the attacker has a prior (distribution) of acceptable parameters.
    UncertainBoxKnowledge = 3
    
# Helper function that returns attacker knowledge on generator
def getAttackerKnowledgeOnGenerator(generator, attackerKnowledgeOnGenerator):
    if attackerKnowledgeOnGenerator == AttackerKnowledgeOnGenerator.BlackBoxKnowledge:
        return tapas.threat_models.BlackBoxKnowledge(generator, num_synthetic_records=100, )
    elif attackerKnowledgeOnGenerator == AttackerKnowledgeOnGenerator.NoBoxKnowledge:
        return tapas.threat_models.NoBoxKnowledge(generator, num_synthetic_records=100, )
    elif attackerKnowledgeOnGenerator == AttackerKnowledgeOnGenerator.UncertainBoxKnowledge:
        return tapas.threat_models.UncertainBoxKnowledge(generator, num_synthetic_records=100, )

In [16]:
# Enum for threat models
class LabelInferenceTrainableThreatModel(Enum):
    # This threat model implements a MIA (membership inference attacks) with arbitrary attacker knowledge on data and generator.
    TargetedMIA = 1
    
    # This threat model implements a AIA (attribute inference attacks) with arbitrary attacker knowledge on data and generator.
    TargetedAIA = 2

# Helper function that returns threat model from the input parameters
def getThreatModel(data_knowledge, target_index, sdg_knowledge, threatModel):
    if threatModel == LabelInferenceTrainableThreatModel.TargetedMIA:
        return tapas.threat_models.TargetedMIA(attacker_knowledge_data=data_knowledge,
                        target_record=dataset.get_records([target_index]),
                        attacker_knowledge_generator=sdg_knowledge,
                        generate_pairs=True,
                        replace_target=True,
                        iterator_tracker=tqdm.tqdm)
    elif threatModel == LabelInferenceTrainableThreatModel.TargetedAIA:
        return tapas.threat_models.TargetedAIA(attacker_knowledge_data=data_knowledge,
                        target_record=dataset.get_records([target_index]),
                        attacker_knowledge_generator=sdg_knowledge,
                        generate_pairs=True,
                        replace_target=True,
                        iterator_tracker=tqdm.tqdm)

In [17]:
# Enum for attacks
class Attack(Enum):
    # Shadow-modelling Attack: Simulates the dataset generation process, using auxiliary info available to the attacker, and train a classifier to predict a property of the training dataset from the synethic dataset
    ShadowModellingAttack = 1
    
    # Groundhog Attack: The attack introduced by Stadler et al.
    GroundhogAttack = 2
    
    # ProbabilityEstimationAttack: Membership Inference Attack that first estimates a statistical model p_x of the distribution of records in the synthetic data, and then uses p_x(target_record) as score. The intuition is that the distribution of the synthetic data, which is defined by the generator trained on the real data, is more likely to be high for records in the real data.
    ProbabilityEstimationAttack = 3
    
    # SyntheticPredictorAttack: Attribute inference attack that first trains a classifier C on the synthetic data to predict the sensitive value v of a record x, then uses C(target_record) as prediction for the target record.
    SyntheticPredictorAttack = 4
    
    # ClosestDistanceAIA: Attack that finds the closest-record to the target record, and uses the value of the sensitive attribute of that closest-record as answer to the attribute-inference attack
    ClosestDistanceAIA = 5
    
    # ClosestDistanceMIA: Attack that looks for the closest record to a given target in the synthetic data to determine whether the target was in the training set
    ClosestDistanceMIA = 6
    
    # LocalNeighbourhoodAttack: Attack that makes a decision based on records similar to the target record, specifically all records within a sphere of a given radius, for a specific choice of distance
    LocalNeighbourhoodAttack = 7
    
    # DirectLinkage: Attack that checks only whether or not the target is in the generated synthetic dataset or not
    DirectLinkage = 8

# Returns attack for the input args
def getAttack(attack, feature_classifier):
    if attack == Attack.ShadowModellingAttack:
        return tapas.attacks.ShadowModellingAttack(feature_classifier)
    elif attack == Attack.GroundhogAttack:
        return tapas.attacks.GroundhogAttack(feature_classifier)
    elif attack == Attack.ProbabilityEstimationAttack:
        return tapas.attacks.ProbabilityEstimationAttack()
    elif attack == Attack.SyntheticPredictorAttack:
        return tapas.attacks.SyntheticPredictorAttack()
    elif attack == Attack.ClosestDistanceAIA:
        return tapas.attacks.ClosestDistanceAIA()
    elif attack == Attack.ClosestDistanceMIA:
        return tapas.attacks.ClosestDistanceMIA()
    elif attack == Attack.LocalNeighbourhoodAttack:
        return tapas.attacks.LocalNeighbourhoodAttack()
    elif attack == Attack.DirectLinkage:
        return tapas.attacks.DirectLinkage()

In [18]:
# Enum for synthetic data generators
class Generator(Enum):
    # Raw data
    Raw = 1
    # Conditional Tabular Generative Adversarial Network models single table - tabular data distribution for synthetic data generators
    CTGAN = 2
    # Differentially Private Conditional Tabular GAN: Incorporates differential privacy guarantees into conditional tabular generative model
    DPCTGAN = 3
    # Private Aggregation of Teacher Ensembles (PATE) framework applied to GANs; modified framework (which is called PATE-GAN) allows us to tightly bound the influence of any individual sample on the model, resulting in tight differential privacy guarantees and thus an improved performance over models with the same guarantees. 
    PATEGAN = 4
    
# Helper function for getting the generator with the input specs
def getGenerator(generator, epsilon, batch_size, epochs):
    if generator == Generator.Raw:
        return Raw()
    elif generator == Generator.CTGAN:
        return CTGAN(epochs=1)
    elif generator == Generator.DPCTGAN:
        return DPCTGAN(epsilon=epsilon, batch_size=batch_size, epochs=epochs)
    elif generator == Generator.PATEGAN:
        return PATEGAN(epsilon=epsilon, batch_size=batch_size)

In [19]:
# Defining number of records to target for random and outlier
num_targets = 5
    
# Selecting random target record indices
random_index = list(np.random.randint(1, 100, num_targets))

# Creates target indices to target for random and outlier
def createTargetIndices():
    # Selecting outlier target record indices
    model_isoforest = IsolationForest()
    preds = model_isoforest.fit_predict(data.data.iloc[:, 3:])
    outlier_index = list(np.random.choice(np.where(preds == -1)[0], num_targets))

    # Listing out target indices by combining random and outlier indicies
    targets = random_index + outlier_index

    return targets

targets = createTargetIndices()

In [20]:
targets

[84, 70, 70, 90, 66, 491, 73, 57, 491, 550]

In [21]:
# Function that defines the attack by consolidating data knowledge, knowledge on the generator as well as training and testing the attacks by calling the helper functions
def attack(dataset, target_index, generator):
    # Getting attacker knowledge on data
    data_knowledge = getAttackerKnowledgeOnData(dataset, AttackerKnowledgeOnData.AuxiliaryDataKnowledge)
    
    # Getting knowledge on generator
    sdg_knowledge = getAttackerKnowledgeOnGenerator(generator, AttackerKnowledgeOnGenerator.BlackBoxKnowledge)
    
    # Defining the threat model membership inference attack on a random record with attacker goal
    threat_model = tapas.threat_models.TargetedMIA(attacker_knowledge_data=data_knowledge,
                        target_record=dataset.get_records([target_index]),
                        attacker_knowledge_generator=sdg_knowledge,
                        generate_pairs=True,
                        replace_target=True,
                        iterator_tracker=tqdm.tqdm)

    # Initializing an attacker of Groundhog attack with standard parameters
    random_forest = RandomForestClassifier(n_estimators=100)
    feature_set = tapas.attacks.NaiveSetFeature() + tapas.attacks.HistSetFeature() + tapas.attacks.CorrSetFeature()
    feature_classifier = tapas.attacks.FeatureBasedSetClassifier(feature_set, random_forest)
    attacker = getAttack(Attack.GroundhogAttack, feature_classifier)
    
    # Training the attack
    start = time.time()
    try:
        attacker.train(threat_model, num_samples=10)
    except:
        print("An exception occurred while training the threat model.")
    end = time.time()
    print("Training time delta for the attacker: {}".format(end-start))

    # Testing the attack
    start = time.time()
    try:
        summary = threat_model.test(attacker, num_samples=10)
    except:
        print("An exception occurred while testing the threat model.")
    end = time.time()
    print("Testing time delta for the attacker: {}".format(end-start))
    
    try:
        metrics = summary.get_metrics()
    except:
        print("An exception occurred while getting metrics for the summary.")
        
    # Defining metrics for data set of Breast Cancer
    metrics["dataset"] = "Breast Cancer"
    
    print("Metrics: ")
    print(metrics)
    
    return summary, metrics

In [22]:
# Main function that collects metrics and summaries from the attacks conducted on generators and targets
def main():
    # Initializing an empty data frame and all_summaries
    all_metrics = pd.DataFrame()
    all_summaries = []
    
    # Defining an array of generators
    generators = [getGenerator(Generator.Raw, 0, 0, 0),
                  getGenerator(Generator.CTGAN, 0, 0, 1), 
                  getGenerator(Generator.DPCTGAN, 1, 64, 1)]

    # Looping through generators
    for generator in generators: 
        # Looping through targets
        for target in targets: 
            try:
                summ, metr = attack(dataset=data, target_index=target, generator=generator)
                all_summaries.append(summ)
                all_metrics = pd.concat([all_metrics, metr], axis=0, ignore_index=True)
                print(metr.head())
            except Exception:
                continue
    return all_metrics, all_summaries

In [23]:
# Calling the main function and printing all metrics and summaries from training and testing the models
all_metrics, all_summaries = main()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 825.96it/s]


Training time delta for the attacker: 0.23276209831237793


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1673.37it/s]
/Volumes/Data_Repos/Desktop Folders/UCLA/Thesis/privacy-sdg-toolbox/tabular-synthetic-data-privacy-auditing-main/tapas/report/attack_summary.py:250: RuntimeWarning: divide by zero encountered in divide
  return np.log(max(np.max(tp/fp), np.max((1-fp)/(1-tp))))


Testing time delta for the attacker: 0.14879107475280762
Metrics: 
         dataset target_id generator     attack  accuracy  true_positive_rate  \
0  Breast Cancer        84       Raw  Groundhog       0.6                 0.6   

   false_positive_rate  mia_advantage  privacy_gain   auc  effective_epsilon  
0                  0.4            0.2           0.8  0.72                inf  
         dataset target_id generator     attack  accuracy  true_positive_rate  \
0  Breast Cancer        84       Raw  Groundhog       0.6                 0.6   

   false_positive_rate  mia_advantage  privacy_gain   auc  effective_epsilon  
0                  0.4            0.2           0.8  0.72                inf  


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2007.23it/s]


...

 90%|█████████████████████████████████████████████████████████████████████████████████████████          | 9/10 [02:29<00:16, 16.38s/it]/Users/krishnadave/Library/Caches/pypoetry/virtualenvs/tapas-T4rVcRt4-py3.9/lib/python3.9/site-packages/opacus/privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/Users/krishnadave/Library/Caches/pypoetry/virtualenvs/tapas-T4rVcRt4-py3.9/lib/python3.9/site-packages/opacus/privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
/Users/krishnadave/Library/Caches/pypoetry/virtualenvs/tapas-T4rVcRt4-py3.9/lib/python3.9/site-packages/torch/nn/modules/module.py:1053: UserWarning: Using a non-full backward hook when the forward c

Epoch 1, Loss G: -0.0517, Loss D: 0.0004
epsilon is 0.028590312727600813, alpha is 63.0
Epoch 2, Loss G: -0.0525, Loss D: 0.0008
epsilon is 0.3508497978739808, alpha is 19.0
Epoch 3, Loss G: -0.0492, Loss D: 0.0015
epsilon is 0.5118818491933068, alpha is 15.0
Epoch 4, Loss G: -0.0512, Loss D: 0.0039
epsilon is 0.6401869495928957, alpha is 13.0
Epoch 5, Loss G: -0.0493, Loss D: 0.0040
epsilon is 0.7519399509030614, alpha is 12.0
Epoch 6, Loss G: -0.0525, Loss D: -0.0005
epsilon is 0.8505949589561193, alpha is 10.6
Epoch 7, Loss G: -0.0535, Loss D: -0.0019
epsilon is 0.9421075032742222, alpha is 9.9


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:45<00:00, 16.55s/it]

Testing time delta for the attacker: 165.61830806732178
Metrics: 
         dataset target_id         generator     attack  accuracy  \
0  Breast Cancer       550  DP-CTGAN (eps=1)  Groundhog       0.4   

   true_positive_rate  false_positive_rate  mia_advantage  privacy_gain   auc  \
0                 0.2                  0.4           -0.2           1.2  0.42   

   effective_epsilon  
0                inf  
         dataset target_id         generator     attack  accuracy  \
0  Breast Cancer       550  DP-CTGAN (eps=1)  Groundhog       0.4   

   true_positive_rate  false_positive_rate  mia_advantage  privacy_gain   auc  \
0                 0.2                  0.4           -0.2           1.2  0.42   

   effective_epsilon  
0                inf  



/Volumes/Data_Repos/Desktop Folders/UCLA/Thesis/privacy-sdg-toolbox/tabular-synthetic-data-privacy-auditing-main/tapas/report/attack_summary.py:250: RuntimeWarning: divide by zero encountered in divide
  return np.log(max(np.max(tp/fp), np.max((1-fp)/(1-tp))))


In [24]:
# Printing all the collected metrics and summaries 
print(all_metrics)
print(all_summaries)

          dataset target_id         generator     attack  accuracy  \
0   Breast Cancer        84               Raw  Groundhog       0.6   
1   Breast Cancer        70               Raw  Groundhog       0.6   
2   Breast Cancer        70               Raw  Groundhog       0.5   
3   Breast Cancer        90               Raw  Groundhog       0.7   
4   Breast Cancer        66               Raw  Groundhog       0.5   
5   Breast Cancer       491               Raw  Groundhog       0.5   
6   Breast Cancer        73               Raw  Groundhog       0.7   
7   Breast Cancer        57               Raw  Groundhog       0.7   
8   Breast Cancer       491               Raw  Groundhog       0.9   
9   Breast Cancer       550               Raw  Groundhog       0.6   
10  Breast Cancer        84             CTGAN  Groundhog       1.0   
11  Breast Cancer        70             CTGAN  Groundhog       1.0   
12  Breast Cancer        70             CTGAN  Groundhog       0.9   
13  Breast Cancer   

## Results and Plots

### Description
Contains utilities to generate results and produce plots/reports to compare generators of random targets and compare attacks for different target record types of random and outlier.

In [34]:
# Generates and saves the report of comparison of generators on random targets
def generateReport_ComparisonOfGeneratorsOnRandomTargets():
    num_attacks = all_metrics.shape[0]
    random_indices = [num for i in range(0, num_attacks, num_targets*2) for num in range(i, i+num_targets)]
    report = tapas.report.BinaryLabelAttackReport(all_metrics.iloc[random_indices])
    report.metrics = ["privacy_gain", "auc", "effective_epsilon"]
    report.compare(comparison_column='generator', fixed_pair_columns=['attack', 'dataset'], marker_column='target_id', filepath="../figures_cancer_synthetic_Dec24/cancer_compare_generators")

In [35]:
generateReport_ComparisonOfGeneratorsOnRandomTargets()

/Users/krishnadave/Library/Caches/pypoetry/virtualenvs/tapas-T4rVcRt4-py3.9/lib/python3.9/site-packages/numpy/lib/function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [36]:
# Generates report that compares attack for different target record types of random and outlier
def generateReport_CompareAttacksForDifferentTargetRecordTypes():
    len_generators = 3
    all_metrics['target_type'] = (['Random']*num_targets + ['Outlier']*num_targets) * len_generators
    report = tapas.report.BinaryLabelAttackReport(all_metrics)
    report.metrics = ["privacy_gain", "auc"]
    report.compare(comparison_column='generator', fixed_pair_columns=['attack', 'dataset'], marker_column='target_type', filepath="../figures_cancer_synthetic_Dec24/Cancer_random_versus_outlier")

In [37]:
generateReport_CompareAttacksForDifferentTargetRecordTypes()